In [1]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Deep Learning API for creating Neural Networks (Runs on TensorFlow)
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import math
from keras.utils.vis_utils import plot_model

from sklearn.metrics import accuracy_score

from my_utils.visualization import show_history_graph, show_ensemble_features

# 0 - Data

In [2]:
train_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\train_processed.ftr')
val_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\val_processed.ftr')
cols = list(train_df.columns)

In [10]:
train_raw = pd.read_csv(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\train.csv')

In [16]:
cols = [
    'Age',
    'RoomService',
    'FoodCourt',
    'ShoppingMall',
    'Spa',
    'VRDeck'
]
mean_dict = dict(train_raw[cols].mean())
std_dict = dict(train_raw[cols].std())
max_dict = dict(train_raw[cols].max())
min_dict = dict(train_raw[cols].min())

In [ ]:
path = os.join.path('models')
# with open():

In [7]:
train_df.head()

,is_g734s,CryoSleep,VIP,Europa,Mars,PSO J318.5-22,TRAPPIST-1e,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,segment,Transported
0,0,1,0,0,0,0,1,-1.091157,-0.632300,-0.641470,-0.620091,-0.652532,-0.632048,0,False
1,0,1,0,0,0,0,1,-0.394799,-0.632300,-0.641470,-0.620091,-0.652532,-0.632048,0,False
2,0,0,0,1,0,0,1,0.719374,-0.632300,1.253259,-0.620091,2.046458,2.081764,1,False
3,0,0,0,0,0,0,1,-1.160793,-0.632300,-0.641470,-0.620091,-0.652532,-0.632048,0,True
4,0,0,0,0,1,0,0,-0.951885,2.389482,-0.641470,1.201909,1.310410,-0.632048,2,False


In [8]:
val_df.head()

,is_g734s,CryoSleep,VIP,Europa,Mars,PSO J318.5-22,TRAPPIST-1e,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,segment,Transported
0,0,0,0,0,1,0,1,-0.581879,0.509857,-0.643205,2.070229,1.321039,-0.623545,2,True
1,0,1,0,1,0,0,1,-1.915467,-0.617751,-0.643205,-0.579595,-0.656205,-0.623545,0,False
2,0,0,0,1,0,0,0,-0.792445,-0.617751,2.107556,-0.579595,1.560732,1.529699,1,True
3,0,0,0,0,1,0,1,0.330576,1.886978,-0.643205,1.591363,-0.656205,0.688500,2,False
4,0,1,0,1,0,0,0,-0.230935,-0.617751,-0.643205,-0.579595,-0.656205,-0.623545,0,True


In [3]:
cols

['is_g734s',
 'CryoSleep',
 'VIP',
 'Europa',
 'Mars',
 'PSO J318.5-22',
 'TRAPPIST-1e',
 'Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'segment',
 'Transported']

In [3]:
x_train = train_df[cols[:-1]].to_numpy()
x_val = val_df[cols[:-1]].to_numpy()
y_train = train_df[cols[-1]].astype(float).to_numpy()
y_val = val_df[cols[-1]].astype(float).to_numpy()

# 1 - Baseline model
* Train and val accuracy are almost the same (~0.78)
* It means that the model in under-fitted

In [4]:
def get_accuracy(x, y, name, model):
    y_pred = model.predict(x)
    y_pred = [1 if y>=0.5 else 0 for y in y_pred.reshape(1,-1)[0]]
    acc = accuracy_score(y, y_pred)
    print(f'Accuracy on {name} data: {round(acc, 3)}')

In [5]:
layers = [
  Dense(4, activation=keras.activations.relu), # hidden layer 1, ReLU activation
  Dense(4, activation=keras.activations.relu), # hidden layer 2, ReLU activation
  Dense(1, activation=keras.activations.sigmoid) # ouput layer, sigmoid activation
]

base_model = keras.Sequential(layers)

In [6]:
base_model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [7]:
history = base_model.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_val, y_val)
)

Epoch 1/100
219/219 [==============================] - 1s 3ms/step - loss: 0.7033 - accuracy: 0.5563 - val_loss: 0.6399 - val_accuracy: 0.6678
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.7300 - val_loss: 0.5941 - val_accuracy: 0.7488
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5709 - accuracy: 0.7612 - val_loss: 0.5678 - val_accuracy: 0.7565
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5491 - accuracy: 0.7685 - val_loss: 0.5484 - val_accuracy: 0.7665
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5360 - accuracy: 0.7686 - val_loss: 0.5366 - val_accuracy: 0.7671
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5269 - accuracy: 0.7727 - val_loss: 0.5273 - val_accuracy: 0.7695
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5202 - accuracy: 0.7742 - val_loss: 0.5223 - val_accuracy: 0.7707

In [8]:
get_accuracy(x_train, y_train, 'train', base_model)

219/219 [==============================] - 1s 2ms/step
Accuracy on train data: 0.788


In [9]:
get_accuracy(x_val, y_val, 'train', base_model)

53/53 [==============================] - 0s 3ms/step
Accuracy on train data: 0.785


In [10]:
history_df = pd.DataFrame(history.history)

show_history_graph(history_df, 'loss', 'val_loss')
show_history_graph(history_df, 'accuracy', 'val_accuracy')

# 2 - More layers, more neurons, more dropout!
* More layers with more neurons and Dropout actions got better model
* Both train and val accuracy is near ~0.80

In [11]:
relu = keras.activations.relu
layers = [
    Dense(512, activation=relu),
    Dropout(0.4),
    Dense(256, activation=relu),
    Dropout(0.4),
    Dense(128, activation=relu),
    Dropout(0.3),
    Dense(64, activation=relu),
    Dropout(0.2),
    Dense(32, activation=relu),
    Dropout(0.15),
    Dense(16, activation=relu),
    Dropout(0.1),
    Dense(8, activation=relu),
    Dense(4, activation=relu),
    Dense(1, activation=keras.activations.sigmoid)
]

model_2 = keras.Sequential(layers)

In [12]:
model_2.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=20,
    verbose=0,
    restore_best_weights=True
)
callbacks = [
    early_stopping_callback
]

In [13]:
history = model_2.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)

Epoch 1/100
219/219 [==============================] - 4s 11ms/step - loss: 0.5888 - accuracy: 0.7313 - val_loss: 0.4922 - val_accuracy: 0.7766
Epoch 2/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4991 - accuracy: 0.7743 - val_loss: 0.4602 - val_accuracy: 0.7843
Epoch 3/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4833 - accuracy: 0.7762 - val_loss: 0.4523 - val_accuracy: 0.7914
Epoch 4/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4696 - accuracy: 0.7827 - val_loss: 0.4534 - val_accuracy: 0.7896
Epoch 5/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4689 - accuracy: 0.7837 - val_loss: 0.4501 - val_accuracy: 0.7926
Epoch 6/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4603 - accuracy: 0.7903 - val_loss: 0.4499 - val_accuracy: 0.7920
Epoch 7/100
219/219 [==============================] - 2s 9ms/step - loss: 0.4567 - accuracy: 0.7889 - val_loss: 0.4427 - val_accuracy: 0.797

In [14]:
get_accuracy(x_train, y_train, 'train', model_2)

219/219 [==============================] - 1s 3ms/step
Accuracy on train data: 0.8


In [15]:
get_accuracy(x_val, y_val, 'val', model_2)

53/53 [==============================] - 0s 4ms/step
Accuracy on val data: 0.798


In [16]:
history_df = pd.DataFrame(history.history)

show_history_graph(history_df, 'loss', 'val_loss')
show_history_graph(history_df, 'accuracy', 'val_accuracy')

## Save model

In [17]:
path = os.path.join(
    os.getcwd(),
    'models',
    'neural_model'
)
model_2.save(path)

INFO:tensorflow:Assets written to: c:\PROJEKTY\KAGGLE\SpaceshipTytanic\models\neural_model\assets


# 3 - Add new ensembe feature

In [ ]:
train_ensemble_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_train_df.ftr')
val_ensemble_df = pd.read_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_val_df.ftr')

In [ ]:
features = ['neural', 'lgbm', 'ada_boost', 'svc']

train_ensemble_feature = model_2.predict(x_train)
train_ensemble_df['neural'] = train_ensemble_feature
train_ensemble_df['mean'] = train_ensemble_df[features].mean(axis=1)

val_ensemble_feature = model_2.predict(x_val)
val_ensemble_df['neural'] = val_ensemble_feature
val_ensemble_df['mean'] = val_ensemble_df[features].mean(axis=1)

In [ ]:
cols = ['neural', 'lgbm', 'ada_boost', 'svc', 'mean', 'y']
train_ensemble_df[cols].to_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_train_df.ftr')
val_ensemble_df[cols].to_feather(r'C:\PROJEKTY\KAGGLE\SpaceshipTytanic\data\ensemble_val_df.ftr')

In [ ]:
features = ['neural', 'lgbm', 'ada_boost', 'svc', 'mean']
show_ensemble_features(train_ensemble_df, features, 'train')